In [1]:
import os

import torch
from torch.utils import data
from torchvision.datasets.folder import pil_loader
from torchvision import tv_tensors

from helpers.utils import load_json

class_map = {
    "bus": 1,
    "car": 2,
    "motor": 3,
    "person": 4,
    "rider": 5,
    "traffic light": 6,
    "traffic sign": 7,
    "bike": 8,
    "truck": 9,
}

class BDDDataset(data.Dataset):
    def __init__(self, root, train=True, transform=None):
        self.root = root
        self.train = train
        self.transform = transform
        self.samples = None
        self.prepare()

    def prepare(self):
        self.samples = []

        if self.train:
            annotations = load_json(os.path.join(self.root, "labels/bdd100k_labels_images_train.json"))
            annotations = annotations[:5000]
            image_dir = os.path.join(self.root, "images/100k/train")
        else:
            annotations = load_json(os.path.join(self.root, "labels/bdd100k_labels_images_val.json"))
            # annotations = annotations[:5000]
            image_dir = os.path.join(self.root, "images/100k/val")

        for (idx, ann) in enumerate(annotations):
            ## filter instances of "lane" and "drivable_area", because they have poly2d instead of box2d
            invalid_idxs = [i for i, x in enumerate(ann["labels"]) if x["category"] in ["lane", "drivable area", "train"]]
            if len(invalid_idxs) == len(ann["labels"]):
                continue
            
            ann["labels"] = [ann["labels"][i] for i in range(len(ann["labels"])) if i not in invalid_idxs]
            
            target = {}
            target["boxes"] = [ann['labels'][i]['box2d'] for i in range(len(ann['labels']))]
            target["boxes"] = [[box["x1"], box["y1"], box["x2"], box["y2"]] for box in target["boxes"]]
            target["labels"] = [class_map[ann['labels'][i]['category']] for i in range(len(ann['labels']))]
            target["image_id"] = idx + 1
            target["area"] = [(box[3] - box[1]) * (box[2] - box[0]) for box in target["boxes"]]
            target["iscrowd"] = [0 for _ in target["boxes"]]
            # no mask        
            
            image_path = os.path.join(image_dir, ann["name"])
                        
            if os.path.exists(image_path):
                self.samples.append((image_path, target))
            else:
                raise FileNotFoundError

    def __getitem__(self, index):
        image_path, annotation = self.samples[index]

        image = pil_loader(image_path)

        if self.transform is not None:
            image = self.transform(image)
            
        # image = tv_tensors.Image(image)
        target = {}
        target["boxes"] = torch.tensor(annotation["boxes"], dtype=torch.float).clone().detach()
        target["labels"] = torch.tensor(annotation["labels"], dtype=torch.int64).clone().detach()
        target["area"] = torch.tensor(annotation["area"], dtype=torch.float).clone().detach()
        target["iscrowd"] = torch.tensor(annotation["iscrowd"], dtype=torch.int64).clone().detach()
        target["image_id"] = annotation["image_id"]

        return image, target

    def __len__(self):
        return len(self.samples)

def custom_collate_fn(batch):
    images, annotations = zip(*batch)
    images = data.dataloader.default_collate(images)
    annotations = list(annotations)
    return images, annotations


In [2]:
from torchvision import transforms

# transform = transforms.Compose([transforms.Resize(64), transforms.ToTensor()])
transform = transforms.Compose([transforms.ToTensor()])
loader_train = data.DataLoader(
    BDDDataset("../data/bdd100k", transform=transform), batch_size=32, shuffle=True, num_workers=10, collate_fn=custom_collate_fn
)

loader_val = data.DataLoader(
    BDDDataset("../data/bdd100k", transform=transform, train=False), batch_size=32, shuffle=False, num_workers=10, collate_fn=custom_collate_fn
)

In [ ]:
# images, annotations = next(iter(loader_train))
# model.train()
# output = model(images, annotations)

In [ ]:
# ## visualize the bboxes 

# import matplotlib.pyplot as plt
# import matplotlib.patches as patches

# def draw_boxes(image, annotation):
#     """
#     Draws bounding boxes on the image.
#     :param image: PIL image
#     :param annotation: Annotation data for the image
#     :return: Image with bounding boxes
#     """
#     # Convert PIL Image to a matplotlib object
#     fig, ax = plt.subplots(1)
#     ax.imshow(image)

#     for box in annotation['boxes']:
#         rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], linewidth=1, edgecolor='r', facecolor='none')
#         ax.add_patch(rect)

#     return fig


# for i, (images, annotations) in enumerate(loader_val):
#     if i == 0:  # Visualize the first batch
#         for j in range(len(images)):
#             image = transforms.functional.to_pil_image(images[j])
#             annotation = annotations[j]
#             fig = draw_boxes(image, annotation)
#             plt.show()
#         break


In [ ]:
# vals = load_json(os.path.join("../bdd100k", "labels/bdd100k_labels_images_val.json"))
# classes = []
# for val in vals:
#     for label in val['labels']:
#         classes.append(label['category'])
        
# from collections import Counter
# counts = Counter(classes)
# # hbar
# import matplotlib.pyplot as plt
# plt.barh(list(counts.keys()), list(counts.values()))

In [ ]:
import torchvision
print(torchvision.__file__)

In [ ]:
## do a baseline model
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN_MobileNet_V3_Large_FPN_Weights

model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1)

num_classes = 9 + 1
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


## num of params
print(f"train params / params  = {sum(p.numel() for p in model.parameters() if p.requires_grad)} / {sum(p.numel() for p in model.parameters())}")

In [ ]:
from helpers.engine import train_one_epoch, evaluate

In [ ]:
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(params, lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


model = model.to(device)

In [ ]:

model.train()
for (images, targets) in loader_train:
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in t.items()} for t in targets]
    outputs = model(images, targets)
    break

In [ ]:
model.eval()
for (images, annotations) in loader_train:
    images = list(image.to(device) for image in images)
    annotations = [{k: v.to(device) for k, v in t.items() if isinstance(v, torch.Tensor)} for t in annotations]
    outputs = model(images)
    break

In [ ]:
from torchvision.ops import box_iou
from netcal.metrics import ECE


In [ ]:
ious = box_iou(outputs[0]['boxes'], annotations[0]['boxes'])#.shape
# true_positives = ious.max(dim=-1)
tp_inds = torch.where(ious.max(dim=-1)[0] > 0.5)[0]
matched_inds = ious.max(dim=-1)[1][tp_inds]
probs = outputs[0]['scores'][tp_inds]


probs

In [ ]:
matched_gt_labels = annotations[0]['labels'][matched_inds]
matched_gt_labels

In [ ]:
outputs[0]['boxes'].shape, annotations[0]['boxes'].shape

In [ ]:
# a, b = next(iter(loader_val))
# b[0]

In [ ]:
from helpers.coco_eval import CocoEvaluator
from helpers.coco_utils import get_coco_api_from_dataset
import pickle

# with open('bdd_coco_evaluator.pkl', 'rb') as f:
#     coco_evaluator = pickle.load(f)['coco']
    
iou_types = ["bbox"]
coco = get_coco_api_from_dataset(loader_val.dataset)
coco_evaluator = CocoEvaluator(coco, iou_types)

# with open('bdd_coco_evaluator.pkl', 'wb') as f:
#     pickle.dump({"coco": coco}, f)

In [ ]:
# model.train()
# images, targets = next(iter(loader_train))
# images = list(image.to(device) for image in images)
# targets = [{k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in t.items()} for t in targets]
# loss_dict = model(images, targets)

In [ ]:
import wandb
from datetime import datetime

expt_name = "baseline"
time_stamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
run = wandb.init(project='bml-od', name=f"{expt_name}-{time_stamp}", config={
    "model": "fasterrcnn_mobilenet_v3_large_fpn",
    "dataset": "bdd100k",
    "model_subset": "all",
})

In [ ]:
# let's train it just for 2 epochs
num_epochs = 5

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_metrics = train_one_epoch(model, optimizer, loader_train, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    eval_metrics = evaluate(model, loader_val, device, coco_evaluator)
    break


In [ ]:
evaluate(model, loader_val, device, coco_evaluator)

In [ ]:
images, targets = next(iter(loader_val))

model.train()
outputs = model(images, targets)

In [ ]:
train_metrics.meters['loss']

In [ ]:
eval0.summarize()

In [ ]:
outputs = model(images)

In [ ]:
images, targets = next(iter(loader_train))


In [ ]:
annotations = load_json(os.path.join("../data/bdd100k/", "labels/bdd100k_labels_images_val.json"))



In [ ]:
categories = []
for ann in annotations:
    for label in ann['labels']:
        categories.append(label['category'])
    
from collections import Counter
print(Counter(categories))